In [2]:
!pip install kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"matrikasubedi","key":"a84a0fc56738f34df5e5a05dc85d333a"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

#change the permission
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d ghraycee/kvasir-dataset

100% 4.64G/4.64G [01:05<00:00, 81.0MB/s]
100% 4.64G/4.64G [01:05<00:00, 76.4MB/s]


In [6]:
from zipfile import ZipFile
file_name = "kvasir-dataset.zip"

with ZipFile(file_name,'r') as zip:
 zip.extractall()
 print('Done')

Done


In [0]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential,Model 
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
#from tensorflow.keras.layers.normalization import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [0]:
img_rows,img_cols = 224,224

MobileNet = MobileNet(weights='imagenet',include_top=False,input_shape=(img_rows,img_cols,3))


In [0]:
for layer in MobileNet.layers:
    layer.trainable = False

# Let's print our layers
#for (i,layer) in enumerate(MobileNet.layers):
    #print(str(i),layer.__class__.__name__,layer.trainable)

In [0]:
def addTopModelMobileNet(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    
    top_model = Dense(1024,activation='relu')(top_model)
    
    top_model = Dense(512,activation='relu')(top_model)
    
    top_model = Dense(num_classes,activation='softmax')(top_model)

    return top_model

In [0]:
num_classes = 8

FC_Head = addTopModelMobileNet(MobileNet, num_classes)

model = Model(inputs = MobileNet.input, outputs = FC_Head)

#print(model.summary())

In [0]:
import os
PATH= os.path.join("/content/kvasir-dataset-v2/kvasir-dataset-v2")

In [0]:
image_dataset = ImageDataGenerator(rescale=1./255,validation_split=0.2,height_shift_range=.15,
                                          width_shift_range=.15,
                                          zoom_range=0.5,
                                          vertical_flip=True,
                                          rotation_range=45)

In [0]:
epochs = 25
batch_size = 32

In [66]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = image_dataset.flow_from_directory(
                PATH,  # Source directory for the training images
                target_size=(img_rows, img_cols),
                batch_size=batch_size,
                class_mode='categorical',
                shuffle=True,
                subset='training',
                seed = 52)

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = image_dataset.flow_from_directory(
                PATH, # Source directory for the validation images
                target_size=(img_rows, img_cols),
                batch_size=batch_size,
                class_mode='categorical',
                shuffle=True,
                subset='validation',
                seed = 52)

Found 6400 images belonging to 8 classes.
Found 1600 images belonging to 8 classes.


In [0]:
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

checkpoint = ModelCheckpoint(
                             'kvasir.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(
                          monitor='val_loss',
                          min_delta=0,
                          patience=10,
                          verbose=1,restore_best_weights=True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=5, 
                                            verbose=1, 
                                            factor=0.2, 
                                            min_lr=0.0001)

callbacks = [earlystop,checkpoint,learning_rate_reduction]

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy']
              )

In [0]:
nb_train_samples = 6400
nb_validation_samples = 1600


In [70]:
history = model.fit_generator(
            train_generator,
            steps_per_epoch=nb_train_samples//batch_size,
            epochs=epochs,
            callbacks=callbacks,
            validation_data=validation_generator,
            validation_steps=nb_validation_samples//batch_size)

Epoch 1/25
200/200 [==============================] - ETA: 0s - loss: 0.7029 - accuracy: 0.7252
Epoch 00001: val_loss improved from inf to 0.52001, saving model to kvasir.h5
200/200 [==============================] - 484s 2s/step - loss: 0.7029 - accuracy: 0.7252 - val_loss: 0.5200 - val_accuracy: 0.7925 - lr: 0.0010
Epoch 2/25
200/200 [==============================] - ETA: 0s - loss: 0.4895 - accuracy: 0.7939
Epoch 00002: val_loss did not improve from 0.52001
200/200 [==============================] - 490s 2s/step - loss: 0.4895 - accuracy: 0.7939 - val_loss: 0.6578 - val_accuracy: 0.7688 - lr: 0.0010
Epoch 3/25
200/200 [==============================] - ETA: 0s - loss: 0.4474 - accuracy: 0.8170
Epoch 00003: val_loss improved from 0.52001 to 0.50242, saving model to kvasir.h5
200/200 [==============================] - 490s 2s/step - loss: 0.4474 - accuracy: 0.8170 - val_loss: 0.5024 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 4/25
200/200 [==============================] - ETA: 0s - lo